In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

options = Options()
options.headless = True

browser = webdriver.Chrome(options=options)

In [2]:
def find_map_links(url):
    browser.get(url)
    elem = browser.find_element_by_xpath("//*")
    source = elem.get_attribute('outerHTML')
    
    source = process_data(source)
    
    all_links = find_all_links(source)
    
    predefined_links = ['google.com/maps', 'maps.google.com', 'map.google.com']
    
    map_links = []
    
    for link in all_links:
        if any(defined_links in link for defined_links in predefined_links) and 'https://www.google.com/maps/vt' not in link:
            map_links.append(link)
    
    return check_links((sorted(map_links)))
    #return map_links
    #return all_links

In [3]:
def process_data(text):
    text = text.replace('&amp;', '&')
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('&quot;', '"')
    text = text.replace('",', ' ')
    text = text.replace('")', " ")
    text = text.replace('">', " ")
    text = text.replace('" ', " ")
    text = text.replace('"]', " ")
    text = text.replace('"}', " ")
    text = text.replace('\\', '')
    text = text.replace("')", ' ')
    
    return text

In [4]:
def find_all_links(file):
    links = list(set(re.findall("(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})", file)))
    return links

In [5]:
def check_links(urls):
    for url in urls:
        if (('embed' in url) and (('q=' in url) or ('pb=!' in url))):
            if 'q=' in url:
                return return_embed_address(url)
            elif 'pb=!' in url:
                return return_long_lat_embed(url)
        
        elif 'https://maps.google.com/maps?ll=3D' in url:
            try:
                if len(url.split("https://maps.google.com/maps?ll=3D")[1].split(',')) == 2:
                    return return_long_lat(url)
                continue
            except:
                continue
        
        elif '@' in url:
            try:
                url = url.split('@')[1]
                temp = url.split(',')
                latitude = temp[0]
                longitude = temp[1]

                return latitude, longitude
            except:
                continue
                
        elif 'current+location/' in url:
            try:
                url = url.split('current+location/')[1]
                temp = url.split(',')
                latitude = find_integer(temp[0])
                longitude = find_integer(temp[1])

                return latitude, longitude
            except:
                continue
                
        elif 'q=' in url:
            try:
                return return_embed_address(url)
            except:
                continue
        
        continue
            
    '''for url in urls:
        if 'https://maps.google.com/maps?ll=3D' in url:
            try:
                if len(url.split("https://maps.google.com/maps?ll=3D")[1].split(',')) == 2:
                    return return_long_lat(url)
                continue
            except:
                continue
                
    for url in urls:
        if 'google.com/maps/@' in url:
            url = url.split('@')[1]
            temp = url.split(',')
            latitude = temp[0]
            longitude = temp[1]
            
            return latitude, longitude'''
            
                
    return 0

In [6]:
def return_long_lat_embed(url):
    temp = url.split('!2d')[1].split('!3d')
    longitude = temp[0]
    latitude = temp[1].split('!')[0]
    
    return latitude, longitude

In [7]:
def return_long_lat(url):
    temp = url.split('?ll')[1].split(',')
    latitude = temp[0][3:]
    longitude = temp[1].split('&')[0]
    
    return latitude, longitude

In [8]:
def return_embed_address(url):
    return url.split('q=')[1].split('&')[0].replace('%20',' ').replace('+', ' ').replace('%2C', ',')

In [9]:
def find_integer(text):
    return(list(re.findall("((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+)", text)[0]))[0]

In [10]:
find_map_links('https://www.peebleshomes.com/neighborhood/downing-farm/')

('39.58896736670392', '-84.17152495088204')

In [11]:
find_map_links('https://metrohomebuilders.com/contact-us/')

'107 Weber Blvd N, Naples, FL 34120'

In [12]:
find_map_links('https://1stchoice-realty.com/contact/')

('37.3031304', '-93.4374225')

In [13]:
find_map_links('https://www.affordablehomessupercenter.com/contact-us/')

('35.9407095', '-90.5795876')

In [14]:
find_map_links('https://alfordhomes.com/contact.html')

'17480 Dallas Parkway Suite 217 Dallas, TX 75287'

In [15]:
find_map_links('https://njhomebuilder.com/contact-us/')

('40.18830547939236', '-74.12101638461101')

In [16]:
find_map_links('https://newenergybuilding.com/contact-us/')

('41.44726600014111', '-86.14366520274953')

In [17]:
find_map_links('http://ranchcountry.com/')

'1411 US-90, Sealy, TX 77474, USA'